# Using Sherlock out-of-the-box
This notebook shows how to predict a semantic type for a given table column.
The steps are basically:
- Download files for word embedding and paragraph vector feature extraction (downloads only once) and initialize feature extraction models.
- Extract features from table columns.
- Initialize Sherlock.
- Make a prediction for the feature representation of the column.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [2]:
%env PYTHONHASHSEED=13
%env PYTHONHASHSEED

env: PYTHONHASHSEED=13


'13'

## Initialize feature extraction models

In [3]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:04.833840 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.871755 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.106181 seconds.


[nltk_data] Downloading package punkt to /home/ritvikp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ritvikp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Extract features

In [4]:
data = pd.Series(
    [
        ["Jane Smith", "Lute Ahorn", "Anna James"],
        ["Amsterdam", "Haarlem", "Zwolle"],
        ["Chabot Street 19", "1200 fifth Avenue", "Binnenkant 22, 1011BH"]
    ],
    name="values"
)

In [5]:
data

0                 [Jane Smith, Lute Ahorn, Anna James]
1                         [Amsterdam, Haarlem, Zwolle]
2    [Chabot Street 19, 1200 fifth Avenue, Binnenka...
Name: values, dtype: object

In [6]:
extract_features(
    "../temporary.csv",
    data
)
feature_vectors = pd.read_csv("../temporary.csv", dtype=np.float32)

Extracting Features: 100%|██████████| 3/3 [00:00<00:00, 188.53it/s]

Exporting 1588 column features


In [7]:
feature_vectors

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.114591,0.024789,-0.129655,0.006171,-0.135411,-0.071066,-0.052694,-0.067143,0.087397,-0.144397
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.053379,0.022491,-0.165058,-0.016199,-0.058179,0.008537,-0.045880,0.024186,0.038253,-0.087760
2,1.0,0.0,1.0,0.666667,0.0,2.0,1.0,3.0,-1.5,0.0,...,-0.021885,0.000305,0.047812,0.120439,-0.093610,0.037608,-0.004563,-0.088528,-0.117394,-0.192903


## Initialize Sherlock

In [8]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

2022-09-23 20:58:20.694775: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 20:58:20.705349: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/home/ritvikp/.conda/envs/sherlock/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Predict semantic type for column

In [9]:
predicted_labels = model.predict(feature_vectors, "sherlock")

W0923 20:58:21.040345 46912496407488 ag_logging.py:142] AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x2aabe32844c0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: y contains previously unseen labels: [46]

In [ ]:
predicted_labels

array(['person', 'city', 'address'], dtype=object)